In [1]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.5 MB/s eta 0:00:00
  Using cached pyarrow-14.0.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (3.0 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 5.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.5/386.5 kB 5.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 5.9 MB/s eta 0:00:00a 0:00:01
Using cached pyarrow-14.0.1-cp310-cp310-macosx_11_0_arm64.whl (24.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 4.7 MB/s eta 0:00:00a 0:00:01
   ━

In [2]:
import os.path as op

from datasets import load_dataset

import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint

import numpy as np
import pandas as pd
import torch

from tools import download_dataset, load_dataset_into_to_dataframe, partition_dataset, IMDBDataset

## Load dataset

In [3]:
download_dataset()

data_frame = load_dataset_into_to_dataframe()
partition_dataset(data_frame)

100% | 80.23 MB 

100%|██████████| 50000/50000 [00:15<00:00, 3262.17it/s]


Class distribution:


In [4]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")
df_test = pd.read_csv("test.csv")

## Tokenization

In [5]:
imdb_ds = load_dataset(
    "csv",
    data_files={
        "train": "train.csv",
        "validation": "val.csv",
        "test": "test.csv",
    },
)

print(imdb_ds)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
from transformers import AutoTokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print(f"Tokenizer input max length: {tokenizer.model_max_length}")
print(f"Tokenizer vocabulary size: {tokenizer.vocab_size}")

Tokenizer input max length: 512
Tokenizer vocabulary size: 30522


In [9]:
def tokenize_text(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)

In [10]:
imdb_tokenized = imdb_ds.map(tokenize_text, batched=True, batch_size=None)

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [11]:
del imdb_ds

In [12]:
imdb_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [13]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Setup Dataloaders

In [14]:
train_ds = IMDBDataset(imdb_tokenized, partition_key="train")
val_ds = IMDBDataset(imdb_tokenized, partition_key="validation")
test_ds = IMDBDataset(imdb_tokenized, partition_key="test")

In [15]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=12,
    shuffle=True,
    num_workers=4,
)

val_loader = DataLoader(
    dataset=val_ds,
    batch_size=12,
    num_workers=4,
)
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=12,
    num_workers=4,
)

## Initialize DistilBERT

In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
for param in model.parameters():
    param.requires_grad = False

model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [18]:
# unfreeze 2 last model layers (pre_classifier layer and classifier layer)

for param in model.pre_classifier.parameters():
    param.requires_grad = True

for param in model.classifier.parameters():
    param.requires_grad = True

## Finetuning DistilBERT

In [22]:
import torchmetrics

class LightningModel(L.LightningModule):
    def __init__(self, model, learning_rate=3e-5):
        super().__init__()
        self._learning_rate = learning_rate
        self._model = model

        self._val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        self._test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)

    def forward(self, input_ids, attention_mask, labels):
        return self._model(input_ids, attention_mask=attention_mask, labels=labels)

    def training_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["label"])
        self.log("train_loss", outputs["loss"])
        return outputs["loss"]

    def validation_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["label"])
        self.log("val_loss", outputs["loss"], prog_bar=True)
        logits = outputs["logits"]
        predicted_labels = torch.argmax(logits, 1)
        self._val_acc(predicted_labels, batch["label"])
        self.log("val_acc", self._val_acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["label"])
        logits = outputs["logits"]
        predicted_labels = torch.argmax(logits, 1)
        self._test_acc(predicted_labels, batch["label"])
        self.log("accuracy", self._test_acc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self._learning_rate)
        return optimizer

lt_model = LightningModel(model)

In [20]:
callbacks = [
    ModelCheckpoint(
        save_top_k=1, mode="max", monitor="val_acc"
    )
]
logger = CSVLogger(save_dir="logs/", name="lt_model")

In [23]:
trainer = L.Trainer(
    max_epochs=3,
    callbacks=callbacks,
    accelerator="auto",
    precision="16-mixed",
    devices="auto",
    logger=logger,
    log_every_n_steps=10,
)

trainer.fit(model=lt_model,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader,
            )

Using 16bit Automatic Mixed Precision (AMP)
/Users/alexandragridneva/ProgrammingTools/miniconda3/envs/dl_lightning_ai_course_v2/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:125: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type                                | Params
------------------------------------------------------------------
0 | _model    | DistilBertForSequenceClassification | 67.0 M
1 | _val_acc  | MulticlassAccuracy                  | 0     
2 | _test_acc | MulticlassAccuracy                  | 0     
------------------------------------------------------------------
592 K     Trainable para

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/alexandragridneva/ProgrammingTools/miniconda3/envs/dl_lightning_ai_course_v2/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
/Users/alexandragridneva/ProgrammingTools/miniconda3/envs/dl_lightning_ai_course_v2/lib/python3.10/site-packages/torch/amp/autocast_mode.py:250: User provided device_type of 'cuda', but CUDA is not available. Disabling
/Users/alexandragridneva/ProgrammingTools/miniconda3/envs/dl_lightning_ai_course_v2/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


In [ ]:
trainer.test(lt_model, dataloaders=train_loader, ckpt_path="best")

Restoring states from the checkpoint path at logs/lt_model/version_0/checkpoints/epoch=2-step=8751.ckpt
Loaded model weights from the checkpoint at logs/lt_model/version_0/checkpoints/epoch=2-step=8751.ckpt
/Users/alexandragridneva/ProgrammingTools/miniconda3/envs/dl_lightning_ai_course_v2/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/Users/alexandragridneva/ProgrammingTools/miniconda3/envs/dl_lightning_ai_course_v2/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing: |          | 0/? [00:00<?, ?it/s]

In [ ]:
trainer.test(lt_model, dataloaders=val_loader, ckpt_path="best")

In [ ]:
trainer.test(lt_model, dataloaders=test_loader, ckpt_path="best")

## Finetune all layers (better do it on gpu)

In [ ]:
all_layers_finetune_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

In [ ]:
lt_model2 = LightningModel(all_layers_finetune_model)

trainer = L.Trainer(
    max_epochs=3,
    callbacks=callbacks,
    accelerator="auto",
    precision="16-mixed",
    devices="auto",
    logger=logger,
    log_every_n_steps=10,
)

trainer.fit(model=lt_model2,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader,
            )

In [ ]:
trainer.test(lt_model2, dataloaders=train_loader, ckpt_path="best")

In [ ]:
trainer.test(lt_model2, dataloaders=val_loader, ckpt_path="best")

In [ ]:
trainer.test(lt_model2, dataloaders=test_loader, ckpt_path="best")